In [193]:
import pandas as pd
import json

import os
import string


import text2emotion as te

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords

# use with text2emotion
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [194]:
pd.set_option('display.max_columns', None)

In [195]:
path="./data-extraction/"
df_dict = {}
for root, dirs, files in os.walk(path):
    for file_name in files:
        if file_name.endswith('.jl') or file_name.endswith('.jsonl'):
            file_path = os.path.join(root, file_name)
            name = os.path.splitext(file_name)[0]
            df_dict[name] = pd.read_json(path_or_buf=file_path, lines=True)

In [196]:
df_dict.keys()

dict_keys(['twitter-who', 'twitter_monkeypox', 'insta-monkeypox', 'twitter-drtedros', 'twitter-pfizer', 'tw_hshtag_covid19', 'reddit-monkeypox', 'twitter-flu', 'twitter-trends', 'tw_hshtag_monkeypox', 'twitter-100-monkeypox', 'tw_hshtag_flu', 'twitter-monkeypox', 'twitter-trends-checkpoint', 'tw_hshtag_covid19-checkpoint'])

In [197]:
df_dict["twitter-drtedros"].head(2)

,_type,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,links,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe
0,snscrape.modules.twitter.Tweet,https://twitter.com/FacedFork/status/162811997...,2023-02-21 19:50:26+00:00,Tedros Ghebreyesus - I Need a Vacation #tedros...,Tedros Ghebreyesus - I Need a Vacation #tedros...,1628119979429330945,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,2,7,0,1628119979429330945,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,None,"[{'_type': 'snscrape.modules.twitter.Video', '...",NaN,None,NaN,None,None,None,None,"[tedrosghebreyesus, drtedros, who, worldhealth...",None,None,158.0,NaN
1,snscrape.modules.twitter.Tweet,https://twitter.com/Bravehe57611757/status/162...,2023-02-19 03:11:28+00:00,Es geht schon wieder los!\nStoppt den #Pademie...,Es geht schon wieder los!\nStoppt den #Pademie...,1627143804292587521,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,1627143804292587521,de,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,None,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,None,None,None,"[Pademiewahn, BillGates, Eugeniker, WHO, DrTed...",None,None,28.0,NaN


In [247]:
df = df_dict["twitter-monkeypox"]

In [248]:
df.head()

,_type,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,links,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card,viewCount,vibe
0,snscrape.modules.twitter.Tweet,https://twitter.com/Univadis/status/1628356434...,2023-02-22 11:30:01+00:00,#Monkeypox remains global health emergency #Me...,#monkeypox remains global health emergency #Me...,1628356434218868738,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,1628356434218868738,en,"<a href=""https://prod1.sprinklr.com"" rel=""nofo...",https://prod1.sprinklr.com,Sprinklr Publishing,[{'_type': 'snscrape.modules.twitter.TextLink'...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,None,None,None,"[Monkeypox, MedTwitter]",None,None,6.0,None
1,snscrape.modules.twitter.Tweet,https://twitter.com/sanidadgob/status/16283551...,2023-02-22 11:25:00+00:00,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,1628355168587726851,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,0,0,1628355168587726851,es,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'_type': 'snscrape.modules.twitter.TextLink'...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,None,None,None,"[ViruelaDelMono, Monkeypox]",None,None,975.0,None
2,snscrape.modules.twitter.Tweet,https://twitter.com/redaccionmedica/status/162...,2023-02-22 10:37:42+00:00,🔴 Detectada una nueva forma severa de la virue...,🔴 Detectada una nueva forma severa de la virue...,1628343266893602817,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,1,0,1628343266893602817,es,"<a href=""https://about.twitter.com/products/tw...",https://about.twitter.com/products/tweetdeck,TweetDeck,[{'_type': 'snscrape.modules.twitter.TextLink'...,None,NaN,None,NaN,None,None,None,None,[monkeypox],None,{'_type': 'snscrape.modules.twitter.SummaryCar...,375.0,None
3,snscrape.modules.twitter.Tweet,https://twitter.com/dougshealth80/status/16283...,2023-02-22 10:32:18+00:00,@profchloeorkin Maybe it’s not the #monkeypox ...,@profchloeorkin Maybe it’s not the #monkeypox ...,1628341906764619778,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,1628288384983498752,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,None,NaN,None,1.628288e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,[monkeypox],None,None,8.0,None
4,snscrape.modules.twitter.Tweet,https://twitter.com/ians_india/status/16283406...,2023-02-22 10:27:18+00:00,Deadly #monkeypox virus identified in people w...,Deadly #monkeypox virus identified in people w...,1628340648435933185,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,1628340648435933185,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'_type': 'snscrape.modules.twitter.TextLink'...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,None,None,None,[monkeypox],None,None,154.0,None


In [249]:
# replace certain words on df as a function
def replace_strings(column: str, to_replace: str, value: str, regex: bool) -> pd.DataFrame:
    return df[column].replace(to_replace=to_replace, value=value, regex=regex)

list_to_replace = ["MonkeyPox", "Monkeypox", "MPOX", "Mpox", "mpox"]
wanted_value = "monkeypox"
[replace_strings(column="renderedContent", to_replace=word, value=wanted_value, regex=True) for word in list_to_replace]

# df["renderedContent"] = replace_strings(column="renderedContent", to_replace="Monkeypox", value="monkeypox", regex=True)

[0       #monkeypox remains global health emergency #Me...
 1       💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...
 2       🔴 Detectada una nueva forma severa de la virue...
 3       @profchloeorkin Maybe it’s not the #monkeypox ...
 4       Deadly #monkeypox virus identified in people w...
                               ...                        
 4995    WHO Renames monkeypox Virus To Avoid Stigmatiz...
 4996    Didn't know "Monkey" was a race.\nMonkeying ar...
 4997    WHO says it's using 'monkeypox' to avoid stigm...
 4998    This shit is why this pandemic will never be o...
 4999    World Health Organization is Giving monkeypox ...
 Name: renderedContent, Length: 5000, dtype: object,
 0       #monkeypox remains global health emergency #Me...
 1       💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...
 2       🔴 Detectada una nueva forma severa de la virue...
 3       @profchloeorkin Maybe it’s not the #monkeypox ...
 4       Deadly #monkeypox virus identified in people w...
   

In [250]:
for word in list_to_replace:
    df["renderedContent"] = replace_strings(column="renderedContent", to_replace=word, value="monkeypox", regex=True)

In [251]:
df = df.drop("url", axis=1).drop("_type", axis=1).drop("cashtags", axis=1)

In [252]:
df

,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,links,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,card,viewCount,vibe
0,2023-02-22 11:30:01+00:00,#Monkeypox remains global health emergency #Me...,#monkeypox remains global health emergency #Me...,1628356434218868738,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,1628356434218868738,en,"<a href=""https://prod1.sprinklr.com"" rel=""nofo...",https://prod1.sprinklr.com,Sprinklr Publishing,[{'_type': 'snscrape.modules.twitter.TextLink'...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,None,None,None,"[Monkeypox, MedTwitter]",None,6.0,None
1,2023-02-22 11:25:00+00:00,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,1628355168587726851,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,0,0,1628355168587726851,es,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'_type': 'snscrape.modules.twitter.TextLink'...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,None,None,None,"[ViruelaDelMono, Monkeypox]",None,975.0,None
2,2023-02-22 10:37:42+00:00,🔴 Detectada una nueva forma severa de la virue...,🔴 Detectada una nueva forma severa de la virue...,1628343266893602817,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,1,0,1628343266893602817,es,"<a href=""https://about.twitter.com/products/tw...",https://about.twitter.com/products/tweetdeck,TweetDeck,[{'_type': 'snscrape.modules.twitter.TextLink'...,None,NaN,None,NaN,None,None,None,None,[monkeypox],{'_type': 'snscrape.modules.twitter.SummaryCar...,375.0,None
3,2023-02-22 10:32:18+00:00,@profchloeorkin Maybe it’s not the #monkeypox ...,@profchloeorkin Maybe it’s not the #monkeypox ...,1628341906764619778,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,1628288384983498752,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,None,NaN,None,1.628288e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,[monkeypox],None,8.0,None
4,2023-02-22 10:27:18+00:00,Deadly #monkeypox virus identified in people w...,Deadly #monkeypox virus identified in people w...,1628340648435933185,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,1628340648435933185,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'_type': 'snscrape.modules.twitter.TextLink'...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,None,None,None,[monkeypox],None,154.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2022-11-28 17:47:11+00:00,WHO Renames Monkeypox Virus To Avoid Stigmatiz...,WHO Renames monkeypox Virus To Avoid Stigmatiz...,1597285994218201088,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,1,0,1597285994218201088,en,"<a href=""https://about.twitter.com/products/tw...",https://about.twitter.com/products/tweetdeck,TweetDeck,[{'_type': 'snscrape.modules.twitter.TextLink'...,None,NaN,None,NaN,None,None,None,None,"[monkeypox, WHO]",{'_type': 'snscrape.modules.twitter.SummaryCar...,NaN,None
4996,2022-11-28 17:47:10+00:00,"Didn't know ""Monkey"" was a race.\nMonkeying ar...","Didn't know ""Monkey"" was a race.\nMonkeying ar...",1597285990452047875,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,2,4,0,1597285990452047875,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,None,None,NaN,"{'_type': 'snscrape.modules.twitter.Tweet', 'u...",NaN,None,None,None,None,[monkeypox],None,NaN,None
4997,2022-11-28 17:47:03+00:00,WHO says it's using 'mpox' to avoid stigmatizi...,WHO says it's using 'monkeypox' to avoid stigm...,15972

In [124]:
# df.describe

In [203]:
# for i in df["vibe"].values:
#     if i != None:
#         print(i)

### Translating non-english tweets to get sentiment emotion

In [13]:
!pip install deepl decouple

In [78]:
! pip install contractions spacy tqdm

In [ ]:
import contractions
import spacy
import nltk

In [126]:
import deepl
env = %env

In [25]:
# env["DEEPL_AUTH"]

In [127]:
auth_key = env["DEEPL_AUTH"]
translator = deepl.Translator(auth_key)

In [124]:
df_es = (df[df["lang"] == "es"])

In [131]:
es_test = df_es["renderedContent"][1]

In [135]:
df["lang"].unique()

array(['en', 'es', 'in', 'nl', 'qht', 'qme', 'fr', 'ja', 'it', 'pt', 'de',
       'und', 'sl', 'ca', 'bn', 'fi', 'ru', 'pl', 'ar', 'th', 'bg', 'vi',
       'iw', 'tr', 'hi', 'kn', 'da', 'cy', 'el', 'ko', 'ml', 'zh', 'eu',
       'fa', 'gu', 'pa', 'sv', 'tl', 'ro', 'mr', 'et', 'ta', 'te', 'ur',
       'am'], dtype=object)

In [253]:
import re
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\n', ' ', text)
    return text

In [254]:
df["rawContent"]

0       #Monkeypox remains global health emergency #Me...
1       💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...
2       🔴 Detectada una nueva forma severa de la virue...
3       @profchloeorkin Maybe it’s not the #monkeypox ...
4       Deadly #monkeypox virus identified in people w...
                              ...                        
4995    WHO Renames Monkeypox Virus To Avoid Stigmatiz...
4996    Didn't know "Monkey" was a race.\nMonkeying ar...
4997    WHO says it's using 'mpox' to avoid stigmatizi...
4998    This shit is why this pandemic will never be o...
4999    World Health Organization is Giving Monkeypox ...
Name: rawContent, Length: 5000, dtype: object

In [255]:
df["cleanedContent"] = {}
for idx, value in enumerate(df["rawContent"]):
    # df["rawContent"][i]
    df["cleanedContent"][idx] = remove_special_characters(text=value, remove_digits=False)
df["cleanedContent"]

/var/folders/m_/jddgwd7n0vq6ct665099gkv80000gn/T/ipykernel_83955/2432520749.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df["cleanedContent"] = {}
/var/folders/m_/jddgwd7n0vq6ct665099gkv80000gn/T/ipykernel_83955/2432520749.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cleanedContent"][idx] = remove_special_characters(text=value, remove_digits=False)


0       Monkeypox remains global health emergency MedT...
1        Qu sabes sobre la viruela del mono   Sntomas ...
2        Detectada una nueva forma severa de la viruel...
3       profchloeorkin Maybe its not the monkeypox vir...
4       Deadly monkeypox virus identified in people wi...
                              ...                        
4995    WHO Renames Monkeypox Virus To Avoid Stigmatiz...
4996    Didnt know Monkey was a race Monkeying around ...
4997    WHO says its using mpox to avoid stigmatizing ...
4998    This shit is why this pandemic will never be o...
4999    World Health Organization is Giving Monkeypox ...
Name: cleanedContent, Length: 5000, dtype: object

In [256]:
for word in list_to_replace:
    df["cleanedContent"] = replace_strings(column="cleanedContent", to_replace=word, value="monkeypox", regex=True)

In [257]:
df.head(2)

,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,links,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,card,viewCount,vibe,cleanedContent
0,2023-02-22 11:30:01+00:00,#Monkeypox remains global health emergency #Me...,#monkeypox remains global health emergency #Me...,1628356434218868738,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,0,1628356434218868738,en,"<a href=""https://prod1.sprinklr.com"" rel=""nofo...",https://prod1.sprinklr.com,Sprinklr Publishing,[{'_type': 'snscrape.modules.twitter.TextLink'...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,None,None,None,"[Monkeypox, MedTwitter]",None,6.0,None,monkeypox remains global health emergency MedT...
1,2023-02-22 11:25:00+00:00,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,1628355168587726851,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,0,0,1628355168587726851,es,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'_type': 'snscrape.modules.twitter.TextLink'...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,None,None,None,"[ViruelaDelMono, Monkeypox]",None,975.0,None,Qu sabes sobre la viruela del mono Sntomas ...


In [ ]:
# def translate_all_lang(df):
#     """Run this only on very clean string data, deepl counts every character translated"""
#     for i, val in enumerate(df["lang"]):
#         if val == "en":
#             translation = df["cleanedContent"][i]
#             df["translatedContent"][i] = translation
#         else:
#             translation = translator.translate_text(df["cleanedContent"][i], target_lang="EN-US")
#             df["translatedContent"][i] = translation.text

# from googletrans import Translator

from deep_translator import GoogleTranslator as Translator
# GoogleTranslator(source='auto', target='de').translate("keep it up, you are awesome")

translator = Translator(target="en")

# df["translatedContent"]={}
# def translate_all_lang(df):
#     for i, val in enumerate(df["lang"]):
#         if val == "en":
#             translation = df["cleanedContent"][i]
#             df["translatedContent"][i] = translation
#         else:
#             translation = translator.translate(df["cleanedContent"][i])
#             df["translatedContent"][i] = translation
            
def my_translation(x):
    if x["lang"] == "en":
        return x["cleanedContent"]
    else:
        translation = translator.translate_text(x["cleanedContent"])
        return translation

df["translatedContent"] = df.apply(my_translation, axis=1)
df

# translate_all_lang(df)
# df

In [224]:
df

,date,rawContent,renderedContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,links,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,card,viewCount,vibe,cleanedContent,translatedContent
0,2023-02-22 11:30:01+00:00,#Monkeypox remains global health emergency #Me...,#monkeypox remains global health emergency #Me...,1628356434218868738,0,0,0,0,1628356434218868738,en,"<a href=""https://prod1.sprinklr.com"" rel=""nofo...",https://prod1.sprinklr.com,Sprinklr Publishing,[{'_type': 'snscrape.modules.twitter.TextLink'...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,None,None,None,"[Monkeypox, MedTwitter]",None,6.0,None,monkeypox remains global health emergency MedT...,monkeypox remains global health emergency MedT...
1,2023-02-22 11:25:00+00:00,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,1628355168587726851,1,0,0,0,1628355168587726851,es,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'_type': 'snscrape.modules.twitter.TextLink'...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,None,None,None,"[ViruelaDelMono, Monkeypox]",None,975.0,None,Qu sabes sobre la viruela del mono Sntomas ...,What do you know about monkeypox Symptoms How ...
2,2023-02-22 10:37:42+00:00,🔴 Detectada una nueva forma severa de la virue...,🔴 Detectada una nueva forma severa de la virue...,1628343266893602817,0,0,1,0,1628343266893602817,es,"<a href=""https://about.twitter.com/products/tw...",https://about.twitter.com/products/tweetdeck,TweetDeck,[{'_type': 'snscrape.modules.twitter.TextLink'...,None,NaN,None,NaN,None,None,None,None,[monkeypox],{'_type': 'snscrape.modules.twitter.SummaryCar...,375.0,None,Detectada una nueva forma severa de la viruel...,Detected a new severe form of monkeypox with h...
3,2023-02-22 10:32:18+00:00,@profchloeorkin Maybe it’s not the #monkeypox ...,@profchloeorkin Maybe it’s not the #monkeypox ...,1628341906764619778,0,0,0,0,1628288384983498752,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,None,NaN,None,1.628288e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,[monkeypox],None,8.0,None,profchloeorkin Maybe its not the monkeypox vir...,profchloeorkin Maybe its not the monkeypox vir...
4,2023-02-22 10:27:18+00:00,Deadly #monkeypox virus identified in people w...,Deadly #monkeypox virus identified in people w...,1628340648435933185,0,0,0,0,1628340648435933185,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'_type': 'snscrape.modules.twitter.TextLink'...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,None,None,None,[monkeypox],None,154.0,None,Deadly monkeypox virus identified in people wi...,Deadly monkeypox virus identified in people wi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2022-11-28 17:47:11+00:00,WHO Renames Monkeypox Virus To Avoid Stigmatiz...,WHO Renames monkeypox Virus To Avoid Stigmatiz...,1597285994218201088,0,0,1,0,1597285994218201088,en,"<a href=""https://about.twitter.com/products/tw...",https://about.twitter.com/products/tweetdeck,TweetDeck,[{'_type': 'snscrape.modules.twitter.TextLink'...,None,NaN,None,NaN,None,None,None,None,"[monkeypox, WHO]",{'_type': 'snscrape.modules.twitter.SummaryCar...,NaN,None,WHO Renames monkeypox Virus To Avoid Stigmatiz...,WHO Renames monkeypox Virus To Avoid Stigmatiz...
4996,2022-11-28 17:47:10+00:00,"Didn't know ""Monkey"" was a race.\nMonkeying ar...","Didn't know ""Monkey"" was a race.\nMonkeying ar...",1597285990452047875,1,2,4,0,1597285990452047875,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twit

In [140]:
#df_es = (df[df["lang"] == "es"])

# list of unique value of lang col
# def for entire df, then translate text fn 
# take lang val put it into target_lang, (all in def)
# put eng strings into transCont as well, i guess...

# disabled for now
# translate_all_lang(df)



# df["translatedContent"] = df["renderedContent"]

# result = translator.translate_text(es_test, target_lang="EN-US")
# result.text

/var/folders/m_/jddgwd7n0vq6ct665099gkv80000gn/T/ipykernel_83955/1650995013.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["translatedContent"][i] = translation
/var/folders/m_/jddgwd7n0vq6ct665099gkv80000gn/T/ipykernel_83955/1650995013.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["translatedContent"][i] = translation.text


In [ ]:
# translatedContent from Deepl, this one is very expensive, so keep it as file

In [142]:
#translated_content_monkeypox = df["translatedContent"]

In [144]:
#translated_content_monkeypox.to_json("translated_content_monkeypox.json", orient='records', lines=True)

In [191]:
translated_content_monkeypox = pd.read_json(path_or_buf="translated_content_monkeypox.json", lines=True)

In [225]:
df

,date,rawContent,renderedContent,id,replyCount,retweetCount,likeCount,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,links,media,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,card,viewCount,vibe,cleanedContent,translatedContent
0,2023-02-22 11:30:01+00:00,#Monkeypox remains global health emergency #Me...,#monkeypox remains global health emergency #Me...,1628356434218868738,0,0,0,0,1628356434218868738,en,"<a href=""https://prod1.sprinklr.com"" rel=""nofo...",https://prod1.sprinklr.com,Sprinklr Publishing,[{'_type': 'snscrape.modules.twitter.TextLink'...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,None,None,None,"[Monkeypox, MedTwitter]",None,6.0,None,monkeypox remains global health emergency MedT...,monkeypox remains global health emergency MedT...
1,2023-02-22 11:25:00+00:00,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,💡 ¿Qué sabes sobre la viruela del mono?\n\n📌 S...,1628355168587726851,1,0,0,0,1628355168587726851,es,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'_type': 'snscrape.modules.twitter.TextLink'...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,None,None,None,"[ViruelaDelMono, Monkeypox]",None,975.0,None,Qu sabes sobre la viruela del mono Sntomas ...,What do you know about monkeypox Symptoms How ...
2,2023-02-22 10:37:42+00:00,🔴 Detectada una nueva forma severa de la virue...,🔴 Detectada una nueva forma severa de la virue...,1628343266893602817,0,0,1,0,1628343266893602817,es,"<a href=""https://about.twitter.com/products/tw...",https://about.twitter.com/products/tweetdeck,TweetDeck,[{'_type': 'snscrape.modules.twitter.TextLink'...,None,NaN,None,NaN,None,None,None,None,[monkeypox],{'_type': 'snscrape.modules.twitter.SummaryCar...,375.0,None,Detectada una nueva forma severa de la viruel...,Detected a new severe form of monkeypox with h...
3,2023-02-22 10:32:18+00:00,@profchloeorkin Maybe it’s not the #monkeypox ...,@profchloeorkin Maybe it’s not the #monkeypox ...,1628341906764619778,0,0,0,0,1628288384983498752,en,"<a href=""http://twitter.com/download/iphone"" r...",http://twitter.com/download/iphone,Twitter for iPhone,None,None,NaN,None,1.628288e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,[monkeypox],None,8.0,None,profchloeorkin Maybe its not the monkeypox vir...,profchloeorkin Maybe its not the monkeypox vir...
4,2023-02-22 10:27:18+00:00,Deadly #monkeypox virus identified in people w...,Deadly #monkeypox virus identified in people w...,1628340648435933185,0,0,0,0,1628340648435933185,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,[{'_type': 'snscrape.modules.twitter.TextLink'...,"[{'_type': 'snscrape.modules.twitter.Photo', '...",NaN,None,NaN,None,None,None,None,[monkeypox],None,154.0,None,Deadly monkeypox virus identified in people wi...,Deadly monkeypox virus identified in people wi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2022-11-28 17:47:11+00:00,WHO Renames Monkeypox Virus To Avoid Stigmatiz...,WHO Renames monkeypox Virus To Avoid Stigmatiz...,1597285994218201088,0,0,1,0,1597285994218201088,en,"<a href=""https://about.twitter.com/products/tw...",https://about.twitter.com/products/tweetdeck,TweetDeck,[{'_type': 'snscrape.modules.twitter.TextLink'...,None,NaN,None,NaN,None,None,None,None,"[monkeypox, WHO]",{'_type': 'snscrape.modules.twitter.SummaryCar...,NaN,None,WHO Renames monkeypox Virus To Avoid Stigmatiz...,WHO Renames monkeypox Virus To Avoid Stigmatiz...
4996,2022-11-28 17:47:10+00:00,"Didn't know ""Monkey"" was a race.\nMonkeying ar...","Didn't know ""Monkey"" was a race.\nMonkeying ar...",1597285990452047875,1,2,4,0,1597285990452047875,en,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twit

In [187]:
# Check in googletrans

In [220]:
# !pip install googletrans==3.1.0a0
!pip install -U deep-translator

In [98]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/mother/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [99]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mother/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [100]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/mother/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [175]:
# df["cleanedContent"] = translated_content_monkeypox

In [258]:
df['tokens'] = df["cleanedContent"].apply(word_tokenize)

In [259]:
df['tokens']

0       [monkeypox, remains, global, health, emergency...
1       [Qu, sabes, sobre, la, viruela, del, mono, Snt...
2       [Detectada, una, nueva, forma, severa, de, la,...
3       [profchloeorkin, Maybe, its, not, the, monkeyp...
4       [Deadly, monkeypox, virus, identified, in, peo...
                              ...                        
4995    [WHO, Renames, monkeypox, Virus, To, Avoid, St...
4996    [Didnt, know, Monkey, was, a, race, Monkeying,...
4997    [WHO, says, its, using, monkeypox, to, avoid, ...
4998    [This, shit, is, why, this, pandemic, will, ne...
4999    [World, Health, Organization, is, Giving, monk...
Name: tokens, Length: 5000, dtype: object

In [228]:
all_words = [word for tokens in df['tokens'] for word in tokens]
freq_dist = FreqDist(all_words)

In [229]:
freq_dist.most_common(10)

[('monkeypox', 7457),
 ('the', 2416),
 ('to', 1664),
 ('a', 1281),
 ('of', 1276),
 ('and', 1158),
 ('in', 1010),
 ('de', 833),
 ('for', 826),
 ('is', 707)]

In [230]:
stop_words = set(stopwords.words('english'))
filtered_words = [word for word in all_words if word.lower() not in stop_words]
filtered_freq_dist = FreqDist(filtered_words)

In [231]:
tokens_without_punctuation = [token for token in filtered_words if token not in string.punctuation]
# filtered_freq_dist = FreqDist(tokens_without_punctuation)

In [232]:
additional_stopwords = ["https", "de", "la", "’", "amp", "...", "``", "''", "'s", "der", "Doctor", "en", "que", "Worlds", "Marburg", "health","Health", "world", "Worlds", "die", "und", "doctorwho-worldsapart.com/r/4608/"]

In [233]:
filtered_words = [word for word in tokens_without_punctuation if word.lower() not in additional_stopwords]
filtered_freq_dist = FreqDist(filtered_words)

In [234]:
# run replace on entire df as a function

# scrape related account(s), build a graph to gather correlations of their topics, pfizer et alli
# maybe keep resource lang of each tweet and translate with deepl and then do text2emotion

In [238]:
filtered_freq_dist.most_common(10)

[('monkeypox', 7457),
 ('COVID19', 422),
 ('virus', 377),
 ('ViruelaDelMono', 332),
 ('vaccine', 323),
 ('ViruelaSmica', 303),
 ('HazComunidad', 267),
 ('cases', 258),
 ('HIV', 254),
 ('people', 229)]

In [239]:
test = df['tokens'][0]

test

['monkeypox', 'remains', 'global', 'health', 'emergency', 'MedTwitter']

In [240]:
def analyse_emotion(text):
    return te.get_emotion(text)

df["analyseEmotion"] = df["translatedContent"].apply(analyse_emotion)

In [279]:
emotions[33]
df["rawContent"][33]

'Important thread below- In vulnerable patients #monkeypox is associated with a host of severe adverse outcomes including bowel perforation #surgeons'

In [246]:
emotions = df["analyseEmotion"]
translated_content = df["translatedContent"]
cleaned_content = df["cleanedContent"]

In [283]:
## gather user location

df['location']=df.user.apply(lambda x:x['location'])
df["location"].value_counts()

                         1021
lanoitanretnI             145
United States             138
International             133
Worldwide                  85
                         ... 
Sanity                      1
Bayern, Deutschland         1
Pittsburgh                  1
Manhattan, NY               1
London United Kingdom       1
Name: location, Length: 1335, dtype: int64

In [173]:
import plotly.express as px
import pandas as pd

emotions = df["analyseEmotion"]

# dataframes = []


# # concatenate dataframes
# data = pd.concat(dataframes, axis=0, ignore_index=True)

# # plot scatter plot
# fig = px.scatter(data, x='index', y='Emotion', size='Score', color='Emotion', hover_name='Emotion',
#                  title='Emotion Scores by Location', height=400, width=800)

fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['Emotion', 'Score'] but received: index
 To use the index, pass it in directly as `df.index`.

In [ ]:
mkp_df.reset_index()

In [ ]:
sia = SentimentIntensityAnalyzer()
monkey_pox['sentiment'] = monkey_pox["renderedContent"].apply(lambda x: sia.polarity_scores(x)['compound'])

In [ ]:
monkey_pox['sentiment'].mean()